In [2]:
import pandas as pd

In [3]:
query = "SELECT * FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` WHERE date BETWEEN '20170101' AND '20170131'"

In [4]:
query

"SELECT * FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` WHERE date BETWEEN '20170101' AND '20170131'"

In [5]:
df = pd.read_gbq(query=query, project_id='urm-kpi-br', dialect='standard', dtype={'visitId':object})

Requesting query... ok.
Job ID: ff636fac-6d46-4793-9e30-8caf8bc7a7ca
Query running...
  Elapsed 7.73 s. Waiting...
  Elapsed 8.78 s. Waiting...
  Elapsed 9.81 s. Waiting...
  Elapsed 10.85 s. Waiting...
  Elapsed 11.89 s. Waiting...
  Elapsed 12.9 s. Waiting...
  Elapsed 13.94 s. Waiting...
  Elapsed 15.0 s. Waiting...
  Elapsed 16.01 s. Waiting...
  Elapsed 17.05 s. Waiting...
  Elapsed 18.07 s. Waiting...
  Elapsed 19.09 s. Waiting...
  Elapsed 20.11 s. Waiting...
  Elapsed 21.14 s. Waiting...
Query done.
Processed: 5.4 GB Billed: 5.4 GB
Standard price: $0.03 USD

Retrieving results...
Got 64694 rows.

Total time taken 1092.26 s.
Finished at 2022-09-19 14:50:36.


In [6]:
df.head()

visitorId  visitNumber     visitId  visitStartTime      date  \
0      None            1  1483857382      1483857382  20170107   
1      None            1  1483781643      1483781643  20170107   
2      None            1  1483791860      1483791860  20170107   
3      None            1  1483831862      1483831862  20170107   
4      None            1  1483856596      1483856596  20170107   

                                              totals  \
0  {'timeOnScreen': None, 'transactionRevenue': N...   
1  {'timeOnScreen': None, 'transactionRevenue': N...   
2  {'timeOnScreen': None, 'transactionRevenue': N...   
3  {'timeOnScreen': None, 'transactionRevenue': N...   
4  {'timeOnScreen': None, 'transactionRevenue': N...   

                                       trafficSource  \
0  {'adContent': None, 'isTrueDirect': None, 'med...   
1  {'adContent': None, 'isTrueDirect': True, 'med...   
2  {'adContent': None, 'isTrueDirect': None, 'med...   
3  {'adContent': None, 'isTrueDirect': True, 'med...   
4  {'adContent': None, 'isTrueDirect': None, 'med...   

                                              device  \
0  {'javaEnabled': None, 'operatingSystemVersion'...   
1  {'javaEnabled': None, 'operatingSystemVersion'...   
2  {'javaEnabled': None, 'operatingSystemVersion'...   
3  {'javaEnabled': None, 'operatingSystemVersion'...   
4  {'javaEnabled': None, 'operatingSystemVersion'...   

                                          geoNetwork  \
0  {'networkLocation': 'not available in demo dat...   
1  {'networkLocation': 'not available in demo dat...   
2  {'networkLocation': 'not available in demo dat...   
3  {'networkLocation': 'not available in demo dat...   
4  {'networkLocation': 'not available in demo dat...   

                           customDimensions  \
0  [{'index': 4, 'value': 'North America'}]   
1           [{'index': 4, 'value': 'APAC'}]   
2  [{'index': 4, 'value': 'South America'}]   
3           [{'index': 4, 'value': 'EMEA'}]   
4  [{'index': 4, 'value': 'North America'}]   

                                                hits        fullVisitorId  \
0  [{'promotion': [], 'referer': 'http://siliconv...  1387446501958018844   
1  [{'promotion': [], 'referer': None, 'contentIn...  8993417660422454202   
2  [{'promotion': [], 'referer': 'https://www.goo...  3896365782347437567   
3  [{'promotion': [], 'referer': None, 'contentIn...  0454906255427936283   
4  [{'promotion': [], 'referer': 'https://www.goo...  9435629583163991867   

  userId clientId channelGrouping  socialEngagementType  
0   None     None        Referral  Not Socially Engaged  
1   None     None          Direct  Not Socially Engaged  
2   None     None  Organic Search  Not Socially Engaged  
3   None     None          Direct  Not Socially Engaged  
4   None     None     Paid Search  Not Socially Engaged

In [7]:
df.shape

(64694, 16)

In [8]:
productos_sesion = []
precios_sesion = []

for linea in df.hits:
  productos_hit = []
  precios_hit = []
  for hit in linea:
    for producto in hit['product']:
      productos_hit.append(producto['productSKU'])
      precios_hit.append(producto['productPrice'])
  productos_sesion.append(productos_hit)
  precios_sesion.append(sum(precios_hit))

In [9]:
len(productos_sesion)

64694

In [10]:
productos_sesion[365]

['GGOEYHPA003510',
 'GGOEYDHJ056099',
 'GGOEYHPA003610',
 'GGOEYOLR018699',
 'GGOEYFKQ020699',
 'GGOEYHPB072210',
 'GGOEYDHJ019399',
 'GGOEYOLR080599',
 'GGOEYOBR078599',
 'GGOEYOCR078099',
 'GGOEYOCR077799',
 'GGOEYOCR077399',
 'GGOEGAAX0318',
 'GGOEGAAX0325',
 'GGOEGAAX0317']

In [11]:
precios_sesion[365]

194850000

In [12]:
s = pd.Series(productos_sesion)

In [13]:
from sklearn.preprocessing import MultiLabelBinarizer

In [14]:
mlb = MultiLabelBinarizer()

In [15]:
df_productos = pd.DataFrame(mlb.fit_transform(s), columns=mlb.classes_, index=df.index)

In [16]:
df = df.join(df_productos)

In [17]:
df['precio'] = precios_sesion

In [18]:
df.head()

visitorId  visitNumber     visitId  visitStartTime      date  \
0      None            1  1483857382      1483857382  20170107   
1      None            1  1483781643      1483781643  20170107   
2      None            1  1483791860      1483791860  20170107   
3      None            1  1483831862      1483831862  20170107   
4      None            1  1483856596      1483856596  20170107   

                                              totals  \
0  {'timeOnScreen': None, 'transactionRevenue': N...   
1  {'timeOnScreen': None, 'transactionRevenue': N...   
2  {'timeOnScreen': None, 'transactionRevenue': N...   
3  {'timeOnScreen': None, 'transactionRevenue': N...   
4  {'timeOnScreen': None, 'transactionRevenue': N...   

                                       trafficSource  \
0  {'adContent': None, 'isTrueDirect': None, 'med...   
1  {'adContent': None, 'isTrueDirect': True, 'med...   
2  {'adContent': None, 'isTrueDirect': None, 'med...   
3  {'adContent': None, 'isTrueDirect': True, 'med...   
4  {'adContent': None, 'isTrueDirect': None, 'med...   

                                              device  \
0  {'javaEnabled': None, 'operatingSystemVersion'...   
1  {'javaEnabled': None, 'operatingSystemVersion'...   
2  {'javaEnabled': None, 'operatingSystemVersion'...   
3  {'javaEnabled': None, 'operatingSystemVersion'...   
4  {'javaEnabled': None, 'operatingSystemVersion'...   

                                          geoNetwork  \
0  {'networkLocation': 'not available in demo dat...   
1  {'networkLocation': 'not available in demo dat...   
2  {'networkLocation': 'not available in demo dat...   
3  {'networkLocation': 'not available in demo dat...   
4  {'networkLocation': 'not available in demo dat...   

                           customDimensions   ...   GGOEYHPA003510  \
0  [{'index': 4, 'value': 'North America'}]   ...                0   
1           [{'index': 4, 'value': 'APAC'}]   ...                0   
2  [{'index': 4, 'value': 'South America'}]   ...                0   
3           [{'index': 4, 'value': 'EMEA'}]   ...                0   
4  [{'index': 4, 'value': 'North America'}]   ...                0   

  GGOEYHPA003610 GGOEYHPB072210 GGOEYOBR078599 GGOEYOCR077399 GGOEYOCR077799  \
0              0              0              0              0              0   
1              0              0              0              0              0   
2              0              0              0              0              0   
3              0              0              0              0              0   
4              0              0              0              0              0   

   GGOEYOCR078099  GGOEYOLR018699  GGOEYOLR080599  precio  
0               0               0               0       0  
1               0               0               0       0  
2               0               0               0       0  
3               0               0               0       0  
4               0               0               0       0  

[5 rows x 1126 columns]

In [19]:
visitas = df.drop(['hits','customDimensions'], axis=1)

In [20]:
visitas.head()

visitorId  visitNumber     visitId  visitStartTime      date  \
0      None            1  1483857382      1483857382  20170107   
1      None            1  1483781643      1483781643  20170107   
2      None            1  1483791860      1483791860  20170107   
3      None            1  1483831862      1483831862  20170107   
4      None            1  1483856596      1483856596  20170107   

                                              totals  \
0  {'timeOnScreen': None, 'transactionRevenue': N...   
1  {'timeOnScreen': None, 'transactionRevenue': N...   
2  {'timeOnScreen': None, 'transactionRevenue': N...   
3  {'timeOnScreen': None, 'transactionRevenue': N...   
4  {'timeOnScreen': None, 'transactionRevenue': N...   

                                       trafficSource  \
0  {'adContent': None, 'isTrueDirect': None, 'med...   
1  {'adContent': None, 'isTrueDirect': True, 'med...   
2  {'adContent': None, 'isTrueDirect': None, 'med...   
3  {'adContent': None, 'isTrueDirect': True, 'med...   
4  {'adContent': None, 'isTrueDirect': None, 'med...   

                                              device  \
0  {'javaEnabled': None, 'operatingSystemVersion'...   
1  {'javaEnabled': None, 'operatingSystemVersion'...   
2  {'javaEnabled': None, 'operatingSystemVersion'...   
3  {'javaEnabled': None, 'operatingSystemVersion'...   
4  {'javaEnabled': None, 'operatingSystemVersion'...   

                                          geoNetwork        fullVisitorId  \
0  {'networkLocation': 'not available in demo dat...  1387446501958018844   
1  {'networkLocation': 'not available in demo dat...  8993417660422454202   
2  {'networkLocation': 'not available in demo dat...  3896365782347437567   
3  {'networkLocation': 'not available in demo dat...  0454906255427936283   
4  {'networkLocation': 'not available in demo dat...  9435629583163991867   

    ...   GGOEYHPA003510 GGOEYHPA003610 GGOEYHPB072210 GGOEYOBR078599  \
0   ...                0              0              0              0   
1   ...                0              0              0              0   
2   ...                0              0              0              0   
3   ...                0              0              0              0   
4   ...                0              0              0              0   

   GGOEYOCR077399  GGOEYOCR077799  GGOEYOCR078099  GGOEYOLR018699  \
0               0               0               0               0   
1               0               0               0               0   
2               0               0               0               0   
3               0               0               0               0   
4               0               0               0               0   

   GGOEYOLR080599  precio  
0               0       0  
1               0       0  
2               0       0  
3               0       0  
4               0       0  

[5 rows x 1124 columns]

In [21]:
diccionarios = ['device', 'trafficSource', 'geoNetwork', 'totals']

In [22]:
import json

# Transformando las llaves de los diccionarios en nuevas columnas
for columna in diccionarios:
    visitas = visitas.join(pd.DataFrame([json.loads(json.dumps(linea)) for linea in visitas[columna]]),rsuffix=('_'+ columna))
visitas.drop(diccionarios, axis=1, inplace=True)


# Corrigiendo el formato de las columnas con valores cuantitativos
totals = df.totals[0].keys()
totals = list(totals)
for columna in totals:
    visitas[columna] = pd.to_numeric(visitas[columna])


# Limpiando los datos
visitas.drop('adwordsClickInfo',axis=1,inplace=True)
columna_na = []
for columna in visitas.columns:
    print(columna + ': ' + str(len(visitas[columna].unique())))
    if len(visitas[columna].unique()) == 1:
        columna_na.append(columna)
visitas.drop(columna_na,axis=1,inplace=True)

visitorId: 1
visitNumber: 202
visitId: 63585
visitStartTime: 63640
date: 31
fullVisitorId: 53041
userId: 1
clientId: 1
channelGrouping: 8
socialEngagementType: 1
10 52048: 2
10 52147: 2
10 52211: 2
10 52213: 2
10 52223: 2
10 52225: 2
10 52230: 2
10 52231: 2
10 52232: 2
10 52233: 2
10 52234: 2
10 52237: 2
10 52238: 2
10 52242: 2
10 52246: 2
10 52250: 2
10 52251: 2
9180748: 2
9180749: 2
9180750: 2
9180751: 2
9180752: 2
9180754: 2
9180755: 2
9180756: 2
9180757: 2
9180758: 2
9180759: 2
9180760: 2
9180761: 2
9180764: 2
9180766: 2
9180767: 2
9180769: 2
9180770: 2
9180771: 2
9180778: 2
9180779: 2
9180781: 2
9180782: 2
9180785: 2
9180791: 2
9180792: 2
9180793: 2
9180797: 2
9180801: 2
9180802: 2
9180804: 2
9180806: 2
9180808: 2
9180809: 2
9180811: 2
9180812: 2
9180814: 2
9180815: 2
9180816: 2
9180817: 2
9180818: 2
9180819: 2
9180820: 2
9180822: 2
9180824: 2
9180833: 2
9180834: 2
9180837: 2
9180838: 2
9180840: 2
9180841: 2
9180844: 2
9180850: 2
9180862: 2
9180863: 2
9180875: 2
9180905: 2
9181019

GGOEGAFJ036214: 2
GGOEGAFJ036215: 2
GGOEGAFJ036216: 2
GGOEGAFJ036217: 2
GGOEGAHB057414: 2
GGOEGAHB057416: 2
GGOEGALB034113: 2
GGOEGALB034115: 2
GGOEGALB036513: 2
GGOEGALB036514: 2
GGOEGALB036515: 2
GGOEGALB059913: 2
GGOEGALB059914: 2
GGOEGALB059915: 2
GGOEGALH036714: 2
GGOEGALH036715: 2
GGOEGALH036717: 2
GGOEGALJ034413: 2
GGOEGALJ036413: 2
GGOEGALJ036414: 2
GGOEGALJ036415: 2
GGOEGALJ036416: 2
GGOEGALJ057914: 2
GGOEGALJ073313: 2
GGOEGALJ073314: 2
GGOEGALL060013: 2
GGOEGALL074614: 2
GGOEGALL074615: 2
GGOEGALP034313: 2
GGOEGALP034314: 2
GGOEGALP034316: 2
GGOEGALP036314: 2
GGOEGALP036315: 2
GGOEGALP036317: 2
GGOEGALQ034214: 2
GGOEGALQ034215: 2
GGOEGALQ034217: 2
GGOEGALQ036614: 2
GGOEGALQ036615: 2
GGOEGALQ036616: 2
GGOEGALQ036617: 2
GGOEGAPB057813: 2
GGOEGAPB057814: 2
GGOEGAPB057815: 2
GGOEGAPB057816: 2
GGOEGAPB058212: 2
GGOEGAPB058213: 2
GGOEGAPB058214: 2
GGOEGAPB058215: 2
GGOEGAPB058217: 2
GGOEGAPB058613: 2
GGOEGAPB058614: 2
GGOEGAPB058617: 2
GGOEGAPJ058012: 2
GGOEGAPJ058013: 2
GGOEGAPJ05

In [23]:
visitas.head()

visitNumber     visitId  visitStartTime      date        fullVisitorId  \
0            1  1483857382      1483857382  20170107  1387446501958018844   
1            1  1483781643      1483781643  20170107  8993417660422454202   
2            1  1483791860      1483791860  20170107  3896365782347437567   
3            1  1483831862      1483831862  20170107  0454906255427936283   
4            1  1483856596      1483856596  20170107  9435629583163991867   

  channelGrouping  10 52048  10 52147  10 52211  10 52213      ...       \
0        Referral         0         0         0         0      ...        
1          Direct         0         0         0         0      ...        
2  Organic Search         0         0         0         0      ...        
3          Direct         0         0         0         0      ...        
4     Paid Search         0         0         0         0      ...        

                          region      subContinent  bounces  hits  newVisits  \
0  not available in demo dataset  Northern America      1.0     1        1.0   
1                          Delhi     Southern Asia      1.0     1        1.0   
2  not available in demo dataset     South America      1.0     1        1.0   
3  not available in demo dataset    Western Europe      1.0     1        1.0   
4                     California  Northern America      1.0     1        1.0   

   pageviews  timeOnSite  totalTransactionRevenue  transactionRevenue  \
0        1.0         NaN                      NaN                 NaN   
1        1.0         NaN                      NaN                 NaN   
2        1.0         NaN                      NaN                 NaN   
3        1.0         NaN                      NaN                 NaN   
4        1.0         NaN                      NaN                 NaN   

   transactions  
0           NaN  
1           NaN  
2           NaN  
3           NaN  
4           NaN  

[5 rows x 1142 columns]

In [24]:
visitas.drop('transactionRevenue',axis=1,inplace=True)

In [25]:
visitas.drop('transactions',axis=1,inplace=True)

In [26]:
visitas.totalTransactionRevenue = visitas.totalTransactionRevenue / 1000000

In [27]:
# Creando lista con variables cuantitativas
cuant = list(set(totals) - set(columna_na) - set(['transactionRevenue','transactions']))

#Creando dataframe con los resultados cuantitativos de las sesiones
visitas_totals = visitas.groupby('fullVisitorId',as_index=False)[cuant].sum()

# Última visita
visitas_ultima = visitas.groupby('fullVisitorId',as_index=False)
visitas_ultima = visitas_ultima['visitNumber'].max()

# Combinación entre visitantes y visitas únicas
usuarios_visitas_unicos = visitas.drop_duplicates(subset=['fullVisitorId','visitNumber'])

# Dataframe usuarios con todos los usuarios únicos y su última visita
usuarios = pd.merge(visitas_ultima,usuarios_visitas_unicos,left_on=['fullVisitorId','visitNumber'],
                    right_on=['fullVisitorId','visitNumber'],how='left')

# Primera visita
visitas_primera = visitas.groupby('fullVisitorId',as_index=False)['visitNumber'].min()
visitas_primera.set_index('fullVisitorId',inplace=True)

# Dataframe usuarios con todos los usuarios unicos + su última visita + primera visita
usuarios = usuarios.join(visitas_primera,how='left',on='fullVisitorId',rsuffix='_primera')
usuarios = pd.merge(usuarios,usuarios_visitas_unicos,left_on=['fullVisitorId','visitNumber_primera'],
                    right_on=['fullVisitorId','visitNumber'],how='left',suffixes=('_ultima','_primera'))

# Dataframe usuarios con todos los usuarios unicos + su última visita + primera visita + sumatoria de las columnas cuant
usuarios = pd.merge(usuarios,visitas_totals,left_on=['fullVisitorId'],
                    right_on=['fullVisitorId'],how='left')

# Eliminando totales
for i in cuant:
    usuarios.drop(i+'_primera',axis=1,inplace=True)
    usuarios.drop(i+'_ultima',axis=1,inplace=True)

# Calculando el tiempo entre primera y última visita
usuarios['tiempo_visitas'] = usuarios.visitStartTime_ultima - usuarios.visitStartTime_primera

In [28]:
#Eliminando las columnas Ids
ids = ['fullVisitorId', 'visitId_ultima','visitId_primera']
usuarios.drop(ids,axis=1,inplace=True)

#Creando una variable 'y' con la columna respuesta
y = usuarios.totalTransactionRevenue.copy()
y.fillna(0,inplace=True)
y[y<0] = 0

#Creando una variable X con todas las variables menos la respuesta
X = usuarios.drop('totalTransactionRevenue',axis=1)


#Transformando las variables cualitativas en numeros
from sklearn.preprocessing import LabelEncoder
cuali = usuarios.dtypes[usuarios.dtypes == 'object'].keys()
for col in cuali:
    lbl = LabelEncoder()
    lbl.fit(list(X[col].values.astype('str')))
    X[col] = lbl.transform(list(X[col].values.astype('str')))

/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [29]:
X.shape

(53041, 2271)

In [30]:
X.head()

visitNumber_ultima  visitStartTime_ultima  date_ultima  \
0                   1             1484597865           15   
1                   1             1485675517           27   
2                   1             1483595731            3   
3                   1             1483629067            4   
4                   1             1484513468           14   

   channelGrouping_ultima  10 52048_ultima  10 52147_ultima  10 52211_ultima  \
0                       7                0                0                0   
1                       7                0                0                0   
2                       6                0                0                0   
3                       4                0                0                0   
4                       4                0                0                0   

   10 52213_ultima  10 52223_ultima  10 52225_ultima       ...        \
0                0                0                0       ...         
1                0                0                0       ...         
2                0                0                0       ...         
3                0                0                0       ...         
4                0                0                0       ...         

   metro_primera  networkDomain_primera  region_primera  subContinent_primera  \
0             40                   1075             135                    17   
1             40                   2545             135                     6   
2             36                    892             128                    12   
3              0                   4503              49                    21   
4             40                   4644             135                    13   

   bounces  hits  pageviews  newVisits  timeOnSite  tiempo_visitas  
0      1.0     1        1.0        1.0         0.0               0  
1      1.0     1        1.0        1.0         0.0               0  
2      0.0     5        5.0        1.0        78.0               0  
3      1.0     1        1.0        1.0         0.0               0  
4      1.0     1        1.0        1.0         0.0               0  

[5 rows x 2271 columns]

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [33]:
X_train.shape

(37128, 2271)

In [34]:
X_train.head()

visitNumber_ultima  visitStartTime_ultima  date_ultima  \
29494                   1             1484468235           14   
10410                   1             1484105824            9   
43766                   1             1484106891            9   
29660                   1             1484089259            9   
444                    24             1483421832            1   

       channelGrouping_ultima  10 52048_ultima  10 52147_ultima  \
29494                       4                0                0   
10410                       6                0                0   
43766                       4                0                0   
29660                       6                0                0   
444                         4                0                0   

       10 52211_ultima  10 52213_ultima  10 52223_ultima  10 52225_ultima  \
29494                0                0                0                0   
10410                0                0                0                0   
43766                0                0                0                0   
29660                0                0                0                0   
444                  0                0                0                0   

            ...        metro_primera  networkDomain_primera  region_primera  \
29494       ...                   40                   1181             135   
10410       ...                   35                    892              21   
43766       ...                   35                      0              21   
29660       ...                   35                      0              21   
444         ...                   35                      0              21   

       subContinent_primera  bounces  hits  pageviews  newVisits  timeOnSite  \
29494                    17      1.0     1        1.0        1.0         0.0   
10410                    12      0.0     4        4.0        1.0        63.0   
43766                    12      1.0     1        1.0        1.0         0.0   
29660                    12      1.0     1        1.0        1.0         0.0   
444                      12      0.0    60       45.0        0.0       876.0   

       tiempo_visitas  
29494               0  
10410               0  
43766               0  
29660               0  
444                 0  

[5 rows x 2271 columns]

In [35]:
y_train.head()

29494     0.00
10410     0.00
43766     0.00
29660     0.00
444      39.37
Name: totalTransactionRevenue, dtype: float64

In [36]:
from sklearn.linear_model import LinearRegression

In [37]:
reg = LinearRegression()

In [38]:
reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [39]:
reg_predict = reg.predict(X_test)

In [40]:
reg_predict

array([ -1.27233847,   0.2469517 ,   4.1205727 , ...,  -3.03281811,
         0.30264267, -71.22979185])

In [41]:
from sklearn.metrics import mean_squared_error

In [42]:
import numpy as np

In [43]:
np.sqrt(mean_squared_error(y_test, reg_predict))

214.2716756769518

In [44]:
from sklearn.ensemble import GradientBoostingRegressor

In [45]:
gb = GradientBoostingRegressor()

In [46]:
gb.fit(X_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [47]:
gb_predict = gb.predict(X_test)

In [48]:
gb_predict[gb_predict < 0] = 0

In [49]:
np.sqrt(mean_squared_error(y_test, gb_predict))

40.64338355652716

In [50]:
resultados = pd.DataFrame()
resultados['revenue'] = y_test
resultados['predict'] = gb_predict
resultados['erro'] = gb_predict - y_test

In [51]:
resultados.head()

revenue  predict     erro
429        0.0  0.23687  0.23687
32524      0.0  0.23687  0.23687
44785      0.0  0.23687  0.23687
31742      0.0  0.23687  0.23687
33125      0.0  0.23687  0.23687

In [52]:
resultados[resultados.revenue > 0].head()

revenue     predict       erro
6926     71.39  139.151478  67.761478
51564    34.58   29.836497  -4.743503
28871    71.97   33.335070 -38.634930
19568    79.18   30.024586 -49.155414
17364    29.97    0.248905 -29.721095

In [53]:
from sklearn.dummy import DummyRegressor

In [54]:
dr = DummyRegressor()

In [55]:
dr.fit(X_train, y_train)

DummyRegressor(constant=None, quantile=None, strategy='mean')

In [56]:
dr_predict = dr.predict(X_test)

In [57]:
np.sqrt(mean_squared_error(y_test, dr_predict))

43.006562211198855